In [ ]:
#!/usr/bin/env python3
"""danielsinkin97@gmail.com"""

import matplotlib.pyplot as plt
import numpy as np

from computer_vision.util.images import load_image_as_array, rgb_to_grayscale
from computer_vision.src.transformation import inverse_warp

from ipywidgets import interact, FloatSlider

image: np.ndarray = rgb_to_grayscale(
    load_image_as_array(
        "/Users/danielsinkin/GitHub_private/computer-vision/data/hummingbird.png"
    )
)

canvas_size: tuple[int, int] = (image.shape[0] * 2, image.shape[1] * 2)
canvas: np.ndarray = np.zeros(canvas_size, dtype=np.uint8)

offset_y = (canvas_size[0] - image.shape[0]) // 2
offset_x = (canvas_size[1] - image.shape[1]) // 2

canvas[offset_y : offset_y + image.shape[0], offset_x : offset_x + image.shape[1]] = (
    image
)


@interact(
    dx=FloatSlider(min=-100, max=100, step=1, value=0),
    dy=FloatSlider(min=-100, max=100, step=1, value=0),
    theta=FloatSlider(min=-180, max=180, step=1, value=0),
)
def show_affine_warp(dx=0.0, dy=0.0, theta=0.0) -> None:
    angle_rad = np.radians(theta)
    cos_a = np.cos(angle_rad)
    sin_a = np.sin(angle_rad)

    center_x = canvas.shape[1] / 2
    center_y = canvas.shape[0] / 2

    T = np.array(
        [
            [cos_a, -sin_a, dx + center_x * (1 - cos_a) + center_y * sin_a],
            [sin_a, cos_a, dy + center_y * (1 - cos_a) - center_x * sin_a],
        ],
        dtype=np.float32,
    )

    warped = inverse_warp(canvas, T, canvas.shape)

    plt.imshow(warped, cmap="gray")
    plt.title(f"dx={dx:.0f}, dy={dy:.0f}, θ={theta:.0f}°")
    plt.axis("off")
    plt.show()